In [5]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [6]:
class value:
    def __init__(self,data, _children=(), _oper ='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._oper = _oper
        self.label = label
    def __repr__(self):
        return f"value(data={self.data})"
        
    def __add__(self,other):
        out = value(self.data + other.data, (self,other), '+')
        
        def _backward():
            self.grad += 1*out.grad
            other.grad += 1*out.grad
        out._backward = _backward    

        return out

    
    def __mul__(self, other):
        out = value(self.data * other.data, (self,other), '*' )
        def _backward():
            self.grad += other.data*out.grad
            other.grad+= self.data*out.grad
        out._backward = _backward    

        return out 
        
    
    def tanh(self):
        x= self.data
        t = ((math.exp(2*x)-1)/(math.exp(2*x)+1))
        out = value(t, (self, ), 'tanh')
        def _backward():
            self.grad = (1-t**2)*out.grad
        out._backward = _backward
        return out
    def backward(self):
    
     topo = []
     visited = set()
     def build_topo(v):
       if v not in visited:
         visited.add(v)
         for child in v._prev:
           build_topo(child)
         topo.append(v)
     build_topo(self)
    
     self.grad = 1.0
     for node in reversed(topo):
       node._backward()   

        


        


In [7]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._oper:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._oper, label = n._oper)
      # and connect this node to it
      dot.edge(uid + n._oper, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._oper)

  return dot

In [8]:
#sample imputs
# inputs x1,x2
x1 = value(2.0, label='x1')
x2 = value(0.0, label='x2')
# weights w1,w2
w1 = value(-3.0, label='w1')
w2 = value(1.0, label='w2')
# bias of the neuron
b = value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()
#draw_dot(o)